In [2]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
from io import StringIO
from pandas.api.types import is_numeric_dtype
import requests as req
from bs4 import BeautifulSoup
import json
import unidecode

import sys
import requests as rq
from bs4 import BeautifulSoup as bs
from time import sleep
from time import time
from random import randint
from warnings import warn

In [193]:
op = webdriver.ChromeOptions()

op.add_argument('--headless')

driver = webdriver.Chrome('/Applications/chromedriver', options=op)
driver2 = webdriver.Chrome('/Applications/chromedriver', options=op)

In [4]:
# ONS LATEST INSIGHTS APRIL Wayback machine archive urls
# url = 'http://web.archive.org/cdx/search/cdx?url=https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights&collapse=digest&from=20210707&to=20220501&output=json'
url = 'http://web.archive.org/cdx/search/cdx?url=ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26&collapse=digest&from=20200101&to=20211231&output=json'

urls = rq.get(url).text
parse_url = json.loads(urls) #parses the JSON from urls.

## Extracts timestamp and original columns from urls and compiles a url list.
url_list = []
for i in range(1,len(parse_url)):
    orig_url = parse_url[i][2]
    tstamp = parse_url[i][1]
    waylink = tstamp+'/'+orig_url
    url_list.append(waylink)
## Compiles final url pattern.
final_urls = []
for url in url_list:
    final_urls.append('https://web.archive.org/web/'+url)

In [182]:
d = {'page': [], 'section': [], 'date': [], 'roundup': [], 'bulletin_url': [], 'final_url': [], 'bulletin_mps': []}
df = pd.DataFrame(data=d)
df

,page,section,date,roundup,bulletin_url,final_url,bulletin_mps


In [237]:
def getbulurl(link, n):
    driver2.get(link)
    
    ps = driver2.find_elements_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n}]/section//p")
    
    date = [i.text for i in ps if len(i.text) < 18][0]
    reldate = "Released: " + date.rstrip()
    
    print(reldate)
        
    atags = [i for i in driver2.find_elements_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n}]/section//a[1]")]
    atags.reverse()
    words = ['Back to the top', 'Data']
    for word in words:
        atags = [i for i in atags if (word not in i.text)]
        
    url1 = [i.get_attribute("href") for i in atags][0]
    
    if url1.split("/")[-1] == 'latest':
        print(url1)
        driver2.get(url1)
        urlPrev = driver2.find_element_by_link_text("View previous releases").get_attribute("href")
        
        url = 'null'
        
        for i in range(1, 20):
            driver2.get(urlPrev + "?page=" + str(i) )
            lis = driver2.find_elements_by_xpath('//*[@id="main"]/div[2]/div/div[1]/ul/li')
            listoas = [i.find_element_by_tag_name("a") for i in lis if i.find_element_by_tag_name("p").text == reldate]
            if len(listoas) > 0:
                url = listoas[0].get_attribute("href")
                break
    
        if url == 'null':
                
            urlPrev = 'https://www.ons.gov.uk/economy/economicoutputandproductivity/output/bulletins/economicactivityandsocialchangeintheukrealtimeindicators/previousReleases'

            for i in range(1, 20):
                driver2.get(urlPrev + "?page=" + str(i) )
                lis = driver2.find_elements_by_xpath('//*[@id="main"]/div[2]/div/div[1]/ul/li')
                listoas = [i.find_element_by_tag_name("a") for i in lis if i.find_element_by_tag_name("p").text == reldate]
                if len(listoas) > 0:
                    url = listoas[0].get_attribute("href")
                    break
 
    else:
        url = url1
    
    return url

getbulurl('https://web.archive.org/web/20210126114928/https://www.ons.gov.uk/economy/economicoutputandproductivity/output/articles/ukeconomylatest/2021-01-25', 3)

Released: 26 January 2021
https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/bulletins/earningsandemploymentfrompayasyouearnrealtimeinformationuk/latest


'https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/bulletins/earningsandemploymentfrompayasyouearnrealtimeinformationuk/january2021'

In [238]:
def getru(link, df):

    driver.get(link)    
    s = 1
    
    for n in range(3, 16):
        
        for i in range(0, 4):
        
            n1 = n
            n2 = 1+i

            print('n: ', n, 's: ', s, 'n1: ', n1, 'n2: ', n2)

            if driver.find_elements_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n1}]/section/h2[{n2}]"):

                content = ""

                content = content + driver.find_element_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n1}]/section/h2[{n2}]").text + "\n"

                ps = driver.find_elements_by_xpath(f"//*[@id='main']/div[2]/div[1]/article/div[{n}]/section//p")

                for i, j in enumerate(ps):
                    if (i != 0) & (j.text != '') & (j.text not in ['', 'Back to the top', 'Analysis | Data']):
                        content = content + j.text + "\n"

                if content not in [i for i in df['roundup']]:
                    df = df.append({'page': link, 'section': s, 'roundup': content, 'bulletin_url': getbulurl(link, n), 'bulletin_mps': 'placeholder'}, ignore_index=True)
    
    return df

In [35]:
print(len(final_urls))
final_urls=final_urls[127:]

216


In [143]:
print(len(final_urls))
final_urls[88]

89


'https://web.archive.org/web/20210808215813/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26'

In [153]:
import time 

In [190]:
def testrunO(link):
    print(link)
    driver.get(link)  
    
    time.sleep(5)

    s = 0
    piece = {}
    links = {}
    dates = {}
    uninc = ['Data download', 'XLSX', 'Download', 'user requested data', 'Explore the latest insights', 'Read the latest insights', 'See the latest insights', 'Explore the latest indicators', 'Read the latest indicators', 'See the latest indicators']
    linkwords = ['Read the bulletin', 'Read the latest', 'Latest', 'Read ', 'Analysis', 'Article']
    
    for n in range(2, 16):
        if driver.find_elements_by_xpath(f'//*[@id="main"]/div[2]/div[1]/article/div[{n}]/section/*'):
            
            for el in driver.find_elements_by_xpath(f'//*[@id="main"]/div[2]/div[1]/article/div[{n}]/section/*'):
                                
                if (len(el.text) < 18) & (('2021' in el.text)|('2020' in el.text)):
                    dates[s].append(el.text)
                else:
#                     if el.text == 'Also published recently':
#                         return [piece, links, dates]

                    linky = False
                    if (el.tag_name == 'h3'):
                        s = s+1
                        piece[s] = []
                        links[s] = []
                        dates[s] = []
                        piece[s].append(el.text)

                    if (el.tag_name == 'p') & (el.text not in ['','🠕 Back to the top','Back to the top']):

                        inc = True
                        inc2 = True
                        
                        # Check if contains a link
                        if el.find_elements_by_xpath("a"):
                            for a in el.find_elements_by_xpath("a"):

                                for w in uninc:
                                    if w in a.text:
                                        inc = False

                                for w in linkwords:
                                    if w in a.text:
                                        linky = True

                                if linky:
                                    links[s].append({'text': a.text, 'link': a.get_attribute("href")})

                                elif inc:
                                    piece[s].append(el.text)


                        else:
                            for w in uninc:
                                if w in el.text:                       
                                    inc2 = False
                            if inc2:
                                piece[s].append(el.text)

                    if (el.tag_name == 'ul') & (el.text not in ['','🠕 Back to the top']):
                        for li in el.find_elements_by_xpath('li'):

                            inc = True
                            inc2 = True

                            if li.find_elements_by_xpath("p"):
                                li = li.find_element_by_xpath("p")                        

                            # Check if contains a link
                            if li.find_elements_by_xpath("a"):
                                for w in uninc:
                                    if w in li.find_element_by_xpath("a").text:
                                        inc = False

                                for w in linkwords:
                                    if w in li.find_element_by_xpath("a").text:
                                        linky = True

                                if linky:
                                    links[s].append({'text': li.text, 'link': li.find_element_by_xpath("a").get_attribute("href")})

                                elif inc:
                                    piece[s].append(" - " + li.text)

                            else:
                                for w in uninc:
                                    if w in li.text:
                                        inc2 = False
                                if inc2:
                                    piece[s].append(" - " + li.text)
                
    return [piece, links, dates]

pieces = testrunO(final_urls[88])
print(pieces[0][1])
print(pieces[1][1])
print(pieces[2][1])

https://web.archive.org/web/20210808215813/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26
['Explore by topic']
[]
[]


In [79]:
pieces[0][7][0] # Remove these

'Also published in the last week'

In [115]:
pieces[0][4][0] # Remove these

'Also published recently'

In [151]:
pieces[0][1][0]

'Explore by topic'

In [191]:
def addpiecesandlinks(url, df):
    pieces = testrunO(url)
    for k in pieces[0].keys():
        if pieces[0][k] not in [i for i in df['roundup']]:
            df = df.append({'page': url, 'section': k, 'date': pieces[2][k], 'roundup': pieces[0][k], 'bulletin_url': pieces[1][k], 'final_url': pieces[1][k], 'bulletin_mps': 'placeholder'}, ignore_index=True)
    return df

In [195]:
for i, url in enumerate(final_urls):
    if i>84:
        print(i, url)
        df = addpiecesandlinks(url, df)

85 https://web.archive.org/web/20210628223526/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26
https://web.archive.org/web/20210628223526/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26
86 https://web.archive.org/web/20210630012441/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26
https://web.archive.org/web/20210630012441/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26
87 https://web.archive.org/web/20210701093506/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26
https://web.archive.org/web/20210701093506/https://www.ons.gov.uk/peoplepopu

In [351]:
df = df.iloc[:-5 , :]


In [247]:
# df[df['roundup'][0] == ]
# df = df[df.apply(lambda x: x['roundup'][0] != 'Explore by topic', axis=1)]
# df = df[df.apply(lambda x: x['roundup'][0] != 'Also published recently', axis=1)]
# df = df[df.apply(lambda x: x['roundup'][0] != 'Also published in the last week', axis=1)]
df = df[df.apply(lambda x: x['roundup'][0] != 'Back to the top', axis=1)]


In [248]:
df

,page,section,date,roundup,bulletin_url,final_url,bulletin_mps
0,https://web.archive.org/web/20201109120409/htt...,1.0,[6 November 2020],[Number of COVID-19 infections continues to in...,"[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder
1,https://web.archive.org/web/20201109120409/htt...,2.0,[3 November 2020],[Deaths involving COVID-19 increased for the s...,"[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder
2,https://web.archive.org/web/20201109120409/htt...,3.0,[9 October 2020],[The economy is 9.2% smaller than it was pre-l...,"[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/economy/grossdomesticpr...,placeholder
3,https://web.archive.org/web/20201109120409/htt...,4.0,[5 November 2020],"[Fewer adults are travelling to work, The prop...","[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder
4,https://web.archive.org/web/20201109120409/htt...,5.0,[9 November 2020],[Overseas travel in spring 2020 was down 96% f...,"[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder
...,...,...,...,...,...,...,...
316,https://web.archive.org/web/20210808215813/htt...,4.0,[5 August 2021],[1 in 68 people in the UK experiencing self-re...,[{'text': 'Read the latest bulletin on the pre...,[{'text': 'Read the latest bulletin on the pre...,placeholder
317,https://web.archive.org/web/20210808215813/htt...,5.0,[5 August 2021],[Average UK daily flights up 117% since May 20...,[{'text': 'Read the latest bulletin on economi...,[{'text': 'Read the latest bulletin on economi...,placeholder
318,https://web.archive.org/web/20210808215813/htt...,6.0,[4 August 2021],[9 out of 10 adults tested positive for COVID-...,[{'text': 'Read more in the Coronavirus (COVID...,[{'text': 'Read more in the Coronavirus (COVID...,placeholder
319,https://web.archive.org/web/20210808215813/htt...,7.0,[3 August 2021],"[UK COVID-19 deaths continue to rise, The numb...",[{'text': 'Read the latest bulletin on deaths ...,[{'text': 'Read the latest bulletin on deaths ...,placeholder


In [250]:
def checklatest(urls, date):
    
    if not urls:
        return "ERR: EMPTY URLS"
        
    urls = [i for i in urls if i['text'] not in ['Data']]
    url = urls[-1]['link']
    
    if url == None:
        return "ERR: URL IS NONE"
    bulletinurl = "ERR: NOTHING SELECTED"
    reldate = "Released: " + date.rstrip()
    e = "http" + url.split("http")[-1]
    e = e.replace("https:/www", "https://www")
    print(e)
    if e.split("/")[-1] == 'latest':

        req = rq.get(e).text
        soup = bs(req,'html.parser')
        
        if '404 - ' in soup.find('title').text:
            return "ERR: URL IS 404"
        prevrelurl = "https://www.ons.gov.uk" + [i for i in soup.find_all("a") if i.text =='View previous releases'][0]['href']

        linkseg = 'null'
        for i in range(1, 20):
            req = rq.get( prevrelurl + "?page=" + str(i) ).text
            soup = bs(req,'html.parser')

            if '404 - ' not in soup.find('title').text:


                lis = soup.find("main").find("div", class_="print-links").find_all('li')

                if len([i.find("a") for i in lis if reldate in i.text]) > 0:
                    linkseg = [i.find("a") for i in lis if reldate in i.text][0]['href']
                    break

        if linkseg == 'null':

            if 'Coronavirus and the social impacts' in soup.find('h1').text:
                prevrelurl = "https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/bulletins/coronavirusandthesocialimpactsongreatbritain/previousReleases"

            elif 'Economic activity and social change' in soup.find('h1').text:
                prevrelurl = 'https://www.ons.gov.uk/economy/economicoutputandproductivity/output/bulletins/economicactivityandsocialchangeintheukrealtimeindicators/previousReleases'

            for i in range(1, 20):
                req = rq.get( prevrelurl + "?page=" + str(i) ).text
                soup = bs(req,'html.parser')

                if '404 - ' not in soup.find('title').text:
                    lis = soup.find("main").find("div", class_="print-links").find_all('li')

                    if len([i.find("a") for i in lis if reldate in i.text]) > 0:
                        linkseg = [i.find("a") for i in lis if reldate in i.text][0]['href']
                        break

        if linkseg == 'null':
            belletinurl = "ERR: NO PREVIOUS RELEASE FOR :: " + e + " DATE :: " + date
        else:
            bulletinurl = "https://www.ons.gov.uk" + linkseg

    elif '/releases/' not in e:
        bulletinurl = e

    else:    
        req = rq.get(e).text
        # parse roundup html using beautifulsoup and store in soup
        soup = bs(req,'html.parser')

        cols = soup.find_all('div', class_="col-wrap")

        pubs = [i for i in cols if 'Publications' in i.text]

        if len(pubs)>0:
            pub = pubs[0]
            pubind = cols.index(pub)
            linksect = cols[pubind+1]
            bulletinurl = "https://www.ons.gov.uk" + linksect.find("a")['href']
        else:
            bulletinurl = "ERR: NO PUBLICATION FOR :: " + e + " DATE :: " + date

    bulletinurl = bulletinurl.replace("https://ons", "https://www.ons")

    return bulletinurl
checklatest(df['bulletin_url'][293], df['date'][293][0])

https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19infectionsurveyantibodydatafortheuk/latest


'ERR: NOTHING SELECTED'

In [251]:
for i in df.index:
    if i>292:
        print(i)
        df['final_url'][i] = checklatest(df['bulletin_url'][i], df['date'][i][0])

293
https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19infectionsurveyantibodydatafortheuk/latest


<ipython-input-251-487007610de8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['final_url'][i] = checklatest(df['bulletin_url'][i], df['date'][i][0])


294
https://www.ons.gov.uk/economy/governmentpublicsectorandtaxes/publicsectorfinance/bulletins/publicsectorfinances/latest
295
https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/howpeoplewithavaccinespenttheirtimeoneyearonfromthefirstuklockdown/greatbritainmarch2021
297
https://www.ons.gov.uk/economy/economicoutputandproductivity/output/bulletins/economicactivityandsocialchangeintheukrealtimeindicators/latest
298
https://www.ons.gov.uk//peoplepopulationandcommunity/birthsdeathsandmarriages/livebirths/articles/provisionalbirthsinenglandandwales/2020andquarter1jantomar2021
299
https://www.ons.gov.uk/releases/coronavirusandselfisolationafterbeingincontactwithapositivecaseinengland1to5june2021
300
https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/bulletins/coronaviruscovid19infectionsurveypilot/latest
302
https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing

In [246]:
df['roundup'][130]

['Back to the top']

In [253]:
dfcopy = df.copy()

In [254]:
df

,page,section,date,roundup,bulletin_url,final_url,bulletin_mps
0,https://web.archive.org/web/20201109120409/htt...,1.0,[6 November 2020],[Number of COVID-19 infections continues to in...,"[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder
1,https://web.archive.org/web/20201109120409/htt...,2.0,[3 November 2020],[Deaths involving COVID-19 increased for the s...,"[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder
2,https://web.archive.org/web/20201109120409/htt...,3.0,[9 October 2020],[The economy is 9.2% smaller than it was pre-l...,"[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/economy/grossdomesticpr...,placeholder
3,https://web.archive.org/web/20201109120409/htt...,4.0,[5 November 2020],"[Fewer adults are travelling to work, The prop...","[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder
4,https://web.archive.org/web/20201109120409/htt...,5.0,[9 November 2020],[Overseas travel in spring 2020 was down 96% f...,"[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder
...,...,...,...,...,...,...,...
316,https://web.archive.org/web/20210808215813/htt...,4.0,[5 August 2021],[1 in 68 people in the UK experiencing self-re...,[{'text': 'Read the latest bulletin on the pre...,http://ons.gov.uk/peoplepopulationandcommunity...,placeholder
317,https://web.archive.org/web/20210808215813/htt...,5.0,[5 August 2021],[Average UK daily flights up 117% since May 20...,[{'text': 'Read the latest bulletin on economi...,https://www.ons.gov.uk/economy/economicoutputa...,placeholder
318,https://web.archive.org/web/20210808215813/htt...,6.0,[4 August 2021],[9 out of 10 adults tested positive for COVID-...,[{'text': 'Read more in the Coronavirus (COVID...,https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder
319,https://web.archive.org/web/20210808215813/htt...,7.0,[3 August 2021],"[UK COVID-19 deaths continue to rise, The numb...",[{'text': 'Read the latest bulletin on deaths ...,https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder


In [257]:
df_nourl = df[df.apply(lambda x: x['final_url'][:3] == 'ERR', axis=1)]
df = df[df.apply(lambda x: x['final_url'][:3] != 'ERR', axis=1)]

In [258]:
df

,page,section,date,roundup,bulletin_url,final_url,bulletin_mps
0,https://web.archive.org/web/20201109120409/htt...,1.0,[6 November 2020],[Number of COVID-19 infections continues to in...,"[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder
1,https://web.archive.org/web/20201109120409/htt...,2.0,[3 November 2020],[Deaths involving COVID-19 increased for the s...,"[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder
2,https://web.archive.org/web/20201109120409/htt...,3.0,[9 October 2020],[The economy is 9.2% smaller than it was pre-l...,"[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/economy/grossdomesticpr...,placeholder
3,https://web.archive.org/web/20201109120409/htt...,4.0,[5 November 2020],"[Fewer adults are travelling to work, The prop...","[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder
4,https://web.archive.org/web/20201109120409/htt...,5.0,[9 November 2020],[Overseas travel in spring 2020 was down 96% f...,"[{'text': 'Analysis', 'link': 'https://www.ons...",https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder
...,...,...,...,...,...,...,...
316,https://web.archive.org/web/20210808215813/htt...,4.0,[5 August 2021],[1 in 68 people in the UK experiencing self-re...,[{'text': 'Read the latest bulletin on the pre...,http://ons.gov.uk/peoplepopulationandcommunity...,placeholder
317,https://web.archive.org/web/20210808215813/htt...,5.0,[5 August 2021],[Average UK daily flights up 117% since May 20...,[{'text': 'Read the latest bulletin on economi...,https://www.ons.gov.uk/economy/economicoutputa...,placeholder
318,https://web.archive.org/web/20210808215813/htt...,6.0,[4 August 2021],[9 out of 10 adults tested positive for COVID-...,[{'text': 'Read more in the Coronavirus (COVID...,https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder
319,https://web.archive.org/web/20210808215813/htt...,7.0,[3 August 2021],"[UK COVID-19 deaths continue to rise, The numb...",[{'text': 'Read the latest bulletin on deaths ...,https://www.ons.gov.uk/peoplepopulationandcomm...,placeholder


In [265]:
def getmps(bulletinurl):
    print(bulletinurl)
    
    bulletinurl=bulletinurl.replace('http:///', 'https://www.ons.gov.uk/')
    
    
    bulletinurl = "http" + bulletinurl.split("http")[-1]
    
    # Open page for roundup
    req = rq.get(bulletinurl).text
    # parse roundup html using beautifulsoup and store in soup
    soup = bs(req,'html.parser')
        
    sects = [sect for sect in soup.find_all('section') if sect.find_all('h2') != []]    
    sect = [sect for sect in sects if 'Main points' in sect.find('h2').text]
    
    if len(sect)>0:
                
        sect = sect[0]

        lis = sect.find('ul')
        if lis != None:
            sibling = lis.next_sibling

            while sibling:
                next_sibling = sibling.next_sibling
                sibling.extract()
                sibling = next_sibling
            
        ps = []
        for i in sect:
            if i.name == 'p':
                ps.append(i.text)

            elif i.name == 'ul':
                for j in i.find_all('li'):
                    ps.append(j.text)            

    else:
        return "NO MAIN POINTS"
        
    mps = [i for i in ps if ('Source: Office for National Statistics' not in i) and ('Find out more' not in i) and ('make the website work as well as possible' not in i) and ('Embed this interactive' not in i) and ('xlsx' not in i) and ('cookies' not in i)]
    mainpoints = ""
    for i in mps: 
        mainpoints = mainpoints + i + '\n' 
    
    return mainpoints

print(getmps(df['final_url'][62]))

http:///peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19infectionsinthecommunityinengland/december2020
In England, an estimated 8.7% (95% confidence interval: 8.2% to 9.3%) of people would have tested positive for antibodies against SARS-CoV-2 on a blood test in November, suggesting they had the infection in the past; there is substantial variation in antibody positivity between regions, from 12.8% (95% confidence interval: 11.4% to 14.5%) in London compared with 3.9% (95% confidence interval: 2.8% to 5.3%) in the South West.
In Wales, an estimated 5.5% (95% confidence interval: 3.6% to 8.0%) of people would have tested positive for antibodies against SARS-CoV-2 on a blood test in November, suggesting they had the infection in the past.
In Northern Ireland, an estimated 3.3% (95% confidence interval: 1.6% to 6.0%) of people would have tested positive for antibodies against SARS-CoV-2 on a blood test in November, suggesting they had the 

In [266]:
for i in df.index:
    if i > 61:
        print(i)
        df['bulletin_mps'][i] = getmps(df['final_url'][i])

62
http:///peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19infectionsinthecommunityinengland/december2020
63
https://www.ons.gov.uk/peoplepopulationandcommunity/culturalidentity/ethnicity/articles/coronavirusandthesocialimpactsondifferentethnicgroupsintheuk/2020


<ipython-input-266-ceb516dfa1e9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bulletin_mps'][i] = getmps(df['final_url'][i])


65
https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/bulletins/covid19schoolsinfectionsurveyround1england/november2020
67
https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/bulletins/coronavirustheukeconomyandsocietyfasterindicators/17december2020
69
https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/bulletins/coronaviruscovid19infectionsurveypilot/18december2020
70
https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/bulletins/deathsregisteredweeklyinenglandandwalesprovisional/weekending11december2020
71
https://www.ons.gov.uk/peoplepopulationandcommunity/educationandchildcare/articles/howhascoronaviruscovid19spreadamongstudentsinengland/2020-12-21
72
https://www.ons.gov.uk//peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26
73
https://www.ons.gov.uk/economy/gov

In [268]:
df_mainpoints = df[df.apply(lambda x: x['bulletin_mps'] == 'NO MAIN POINTS', axis=1)]
df = df[df.apply(lambda x: x['bulletin_mps'] != 'NO MAIN POINTS', axis=1)]

In [273]:
df[df.apply(lambda x: len(x['bulletin_mps']) < 300, axis=1)]

,page,section,date,roundup,bulletin_url,final_url,bulletin_mps
30,https://web.archive.org/web/20201125124407/htt...,11.0,[20 November 2020],[People staying at home during second lockdown...,"[{'text': 'Analysis', 'link': 'http://ons.gov....",https://www.ons.gov.uk/peoplepopulationandcomm...,"This week, over the period 11 to 15 November, ..."
38,https://web.archive.org/web/20201128013125/htt...,9.0,[27 November 2020],[Proportion of working adults travelling to wo...,"[{'text': 'Analysis', 'link': 'http://ons.gov....",https://www.ons.gov.uk/peoplepopulationandcomm...,"This week, over the period 18 to 22 November, ..."


In [275]:
df.loc[30]['roundup']

['People staying at home during second lockdown',
 'Under the current lockdown restrictions, people in England are required to stay at home except for specific purposes.',
 'The most common reason to leave home was to shop for food and medicine, reported by 75% of adults who left home in the past seven days in England.',
 'A similar percentage was reported across all three Tiers two weeks ago (Tier 1: 77%, Tier 2: 79%, Tier 3: 76%).',
 'The largest change in the reasons for leaving home in the past seven days when comparing responses from those in Tier 1 two weeks ago and responses for all adults in England during national lockdown, came from adults going out to eat or drink at a restaurant, bar or pub (1% this week), and the percentage of adults who left home to meet up with people in a personal place (4% this week).',
 'This week, 1 in 10 (10%) adults in England who left home in the past seven days met up with people in a public place. A slightly higher percentage was reported by tho

In [276]:
df.loc[30]['bulletin_mps']

'This week, over the period 11 to 15 November, based on adults in Great Britain:\naround half (51%) of working adults reported travelling to work (exclusively and in combination with working from home); this is a decrease from 58% last week\n'

In [278]:
df.to_csv('covidrounduporig.csv', index=False)